In [ ]:
# Install required libraries
!pip install transformers torch torchvision streamlit pyngrok PILLOW


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# This is the code for your Streamlit app, saved as a string
app_code = """
import streamlit as st
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import tempfile
import os

# Setup the processor and model for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Function to generate multiple captions
# Function to generate multiple captions with more diversity
def generate_multiple_captions(image_path, num_captions=3):
    # Open the image
    raw_image = Image.open(image_path).convert("RGB")

    # Preprocess the image and prepare input for the model
    inputs = processor(raw_image, return_tensors="pt")

    captions = []
    for _ in range(num_captions):
        # Generate captions with more diversity
        out = model.generate(**inputs,
                             num_beams=5,
                             num_return_sequences=1,
                             no_repeat_ngram_size=2,
                             temperature=1.0,  # Add randomness
                             top_p=0.9,        # Sampling from top 90% of the distribution
                             top_k=50)         # Sampling from the top 50 words

        caption = processor.decode(out[0], skip_special_tokens=True)

        # To ensure unique captions, we can check if the caption has already been generated
        while caption in captions:
            out = model.generate(**inputs,
                                 num_beams=5,
                                 num_return_sequences=1,
                                 no_repeat_ngram_size=2,
                                 temperature=1.0,
                                 top_p=0.9,
                                 top_k=50)
            caption = processor.decode(out[0], skip_special_tokens=True)

        captions.append(caption)

    return captions


# Streamlit UI
def main():
    st.title("Image Caption Generator")

    # Upload image
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        # Save the uploaded image as a temporary file
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(uploaded_file.read())
            tmp_image_path = tmp_file.name

        # Open the image and display it
        image = Image.open(tmp_image_path)
        st.image(image, caption="Uploaded Image.", use_column_width=True)

        # Generate captions
        num_captions = st.slider("Select the number of captions", 1, 5, 3)
        if st.button("Generate Captions"):
            captions = generate_multiple_captions(tmp_image_path, num_captions)
            for idx, caption in enumerate(captions):
                st.write(f"Caption {idx + 1}: {caption}")

if __name__ == "__main__":
    main()
"""

# Now, we will tell Colab to create a file called 'app.py' and write the above code into it
with open("/content/app.py", "w") as file:
    file.write(app_code)

print("app.py has been saved in Colab!")


app.py has been saved in Colab!


In [ ]:
from pyngrok import ngrok

# Replace this string with your actual ngrok auth token
ngrok.set_auth_token("2mWjetc7OJYIHUM5Wvpl1uDkrxs_24Bzc6kXq5pgcQKPMZbbi")


In [ ]:
import subprocess
from pyngrok import ngrok

# Start the Streamlit app in the background
subprocess.Popen(['streamlit', 'run', '/content/app.py'])

# Open an ngrok tunnel on port 8501 (Streamlit's default port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Streamlit app is running at: NgrokTunnel: "https://801b-34-80-230-203.ngrok-free.app" -> "http://localhost:8501"
